In [1]:
import classy as Class
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import time
import pickle as pkl
import os
from datetime import datetime
from IPython.display import display, clear_output

######################################################
#  Setting up plots
######################################################

%config InlineBackend.figure_format = 'retina'
font = {'size'   : 16, 'family':'STIXGeneral'}
axislabelfontsize='large'
mpl.rc('font', **font)
plt.rcParams["figure.figsize"] = [8.0,6.0]
#plt.rcParams['text.usetex']=True
fsize=16

######################################################
#  Setting fiducial parameters
#  Planck18 TT,TE,EE+lowE+lensing+BAO
######################################################

As = 2.100549e-9
ln10e10As = np.log(1e10*As)

ll_max = 4000

standard_params = {
    "omega_b": 0.0223828,
    "omega_cdm": 0.1201075,
    "n_s": 0.9660499,
    "tau_reio": 0.05430842,
    "ln10^{10}A_s": ln10e10As,
    #"100*theta_s": 1.04,
    "H0": 67.32117
}

common_settings = {'output' : 'tCl,pCl,lCl',
                   'thermodynamics_verbose': 0,
                   'input_verbose': 0,
                   'recombination': 'RECFAST',
                   'lensing': 'yes',
                   'xe_pert_type': 'none',
                   'N_ur': 2.0308,
                   'N_ncdm': 1,
                   'm_ncdm': 0.06,
                   'T_ncdm': 0.71611, #1 species of massive neutrinos
                  }

precision_settings = {}
with open("/Users/gabe/projects/class_h0/cl_ref.pre") as f:
    lines = f.readlines()
    for line in lines:
        line = line.strip()
        if len(line)>0:
            if not line.startswith("#"):
                split = line.split("=")
                precision_settings[split[0].strip()] = float(split[1].strip())

print(precision_settings)


{'tol_ncdm_bg': 1e-10, 'tol_thermo_integration': 1e-05, 'recfast_x_He0_trigger_delta': 0.01, 'recfast_x_H0_trigger_delta': 0.01, 'evolver': 0.0, 'k_min_tau0': 0.002, 'k_max_tau0_over_l_max': 15.0, 'k_step_sub': 0.015, 'k_step_super': 0.0001, 'k_step_super_reduction': 0.1, 'start_small_k_at_tau_c_over_tau_h': 0.0004, 'start_large_k_at_tau_h_over_tau_k': 0.05, 'tight_coupling_trigger_tau_c_over_tau_h': 0.005, 'tight_coupling_trigger_tau_c_over_tau_k': 0.008, 'start_sources_at_tau_c_over_tau_h': 0.006, 'l_max_g': 50.0, 'l_max_pol_g': 25.0, 'l_max_ur': 50.0, 'l_max_ncdm': 50.0, 'l_max_scalars': 10000.0, 'tol_perturbations_integration': 1e-06, 'perturbations_sampling_stepsize': 0.01, 'radiation_streaming_approximation': 2.0, 'radiation_streaming_trigger_tau_over_tau_k': 240.0, 'radiation_streaming_trigger_tau_c_over_tau': 100.0, 'ur_fluid_approximation': 2.0, 'ur_fluid_trigger_tau_over_tau_k': 50.0, 'ncdm_fluid_approximation': 3.0, 'ncdm_fluid_trigger_tau_over_tau_k': 51.0, 'tol_ncdm_synchr

In [ ]:
class_settings = {}
class_settings.update(common_settings)
class_settings.update(standard_params)
class_settings.update(precision_settings)
M = Class.Class()
tic = time.perf_counter()
M.set(class_settings)
M.compute()
toc = time.perf_counter()

Tcmb = M.T_cmb()*10**6 #cmb temp in micro kelvin, because
muK2 = (Tcmb)**2
ll_max_request = 10000
    ######################################################
    # Computing fiducial spectra
    ######################################################
result_dict = {}
result_dict["tt"] =  muK2*M.lensed_cl(ll_max_request)['tt'][2:]
result_dict["te"] =  muK2*M.lensed_cl(ll_max_request)['te'][2:]
result_dict["ee"] =  muK2*M.lensed_cl(ll_max_request)['ee'][2:]
result_dict["pp"] =  muK2*M.lensed_cl(ll_max_request)['pp'][2:]
result_dict["settings"] = "cl_ref.pre"
result_dict["settings"]["s"] = toc-tic

M.struct_cleanup()

In [ ]:
with open("../data/class_ultra_precise.pkl", "wb") as f:
    f.dump(f, result_dict)